# Single Pipe Reline FEA: Plaxis 3D

In [1]:
# import modules
import plxhelper.single_pipe_reline_task as single_pipe_reline
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'

## Specify Parameters

In [2]:
# in-situ soil parameters and geometry
γ_soil_pci = (γ_soil := 125 # pcf
             )/1728
Ms_psi = 1000 # psi
h_cover_in = (h_cover := 15 # ft
             )*12
h_bedding_in = (h_bedding := h_cover/3 # ft
             )*12

# liner pipe geometry
h_ID_in = 44 # in
w_ID_in = 67 # in
t_in = 1.339 # in
h_OD_in = h_ID_in + 2*t_in
w_OD_in = h_OD_in + 2*t_in
h_AVG_in = h_ID_in + t_in
w_AVG_in = h_OD_in + t_in

# depth from structure crown to springline
d_springline_in = h_OD_in - (327.15 # mm
                  /25.4)
# height of soil from grade to springline
h_springline_in = h_cover_in + d_springline_in

# modeled extents
grade_el = h_cover_in + h_AVG_in
water_el = grade_el
h_model_in = h_cover_in + h_AVG_in + h_bedding_in
xmin, ymin, xmax, ymax = (-w_AVG_in*3, -180, w_AVG_in*3, 180)

## Interpolate Hardening Soil Model parameters for given soil type and Ms value

In [3]:
# Springline overburden pressure
print(f"{(σ_z_psi := γ_soil_pci * h_springline_in):.2f} psi")

# Type of soil
backfill_soil_type = ("duncan_selig_interpolated", "SW",  # well graded sand
                      Ms_psi, σ_z_psi)

15.47 psi


## Soil Model for Annular Fill

In [4]:
annular_fill_type = ("linear_elastic_soil", "Grout", 40  # pcf
                     )

## Add soil models, soil layer materials, boreholes

In [5]:
soil_layer_materials_list = [backfill_soil_type]
boreholes_dict = {
    (0,0): dict(layers = [h_model_in], top_el = grade_el, water_table_el = water_el),
}

## Add Live Load Patches: HL-93 Truck and Tandem

In [6]:
# position of live load: center of model, top of grade
xyz_live_load = (0, 0, grade_el)

# list of live loads to be investigated
live_load_list = ["HL93 Tandem Axle", 
                  "HL93 Truck Axle",
                  ]

## Add Lane Load

In [7]:
lane_load = "AASHTO Lane Load"

## Specify and set up parent pipe

In [8]:
h_parent = 61  # in

parent_shape_info_dict = dict(
    segments = [
        dict(SegmentType = "Arc",
             RelativeStartAngle1 = 180,  # deg; 180 because starting at crown
             Radius = 42.125,  # in
             CentralAngle = 84.95,  # deg
            ),
        dict(SegmentType = "Arc",
             Radius = 17.625,  # in
             CentralAngle = 78.70,  # deg
            ),
        dict(SegmentType = "SymmetricExtend",
            ),
        dict(SegmentType = "SymmetricClose",
            ),
    ])

## Specify and set up reline pipe

In [9]:
short_term_reline_type = "plate", "GRPLinerPipe", "34mm", "Short"
long_term_reline_type = "plate", "GRPLinerPipe", "34mm", "Long"

# draw shape starting at crown z elevation
reline_shape_info_dict = dict(
    segments = [
        dict(SegmentType = "Arc",
             RelativeStartAngle1 = 180,  # deg; 180 because starting at crown
             Radius = 33.5 + t_in/2,  # in
             CentralAngle = 85.2,  # deg
            ),
        dict(SegmentType = "Arc",
             Radius = 8.875 + t_in/2,  # in
             CentralAngle = 78.3,  # deg
            ),
        dict(SegmentType = "SymmetricExtend",
            ),
        dict(SegmentType = "SymmetricClose",
            ),
    ])

## Run Single Pipe Reline Routine

In [10]:
tc = single_pipe_reline.task_chain(xmin, ymin, xmax, ymax, grade_el, h_cover_in, h_parent, h_AVG_in, xyz_live_load, lane_load, 
                             parent_shape_info_dict, reline_shape_info_dict, boreholes_dict, 
                             annular_fill_type, soil_layer_materials_list, 
                             short_term_reline_type, long_term_reline_type)

In [11]:
for t in tc.mesh_project[:-1]():
    print(t)

None
None
None
None
z_parent_crown = 53.1695
z_reline_crown) = 45.339
None
